In [1]:

from vllm import LLM, SamplingParams
model_name="/data/2024_llm/sftlab/out_data/sftlab-experiments/test/1-gemma_full-zero3"

tensor_parallel_size = 1
# トークナイザーとモデルの準備
model = LLM(
    model=model_name,
    trust_remote_code=True,
    max_model_len=2000,
    tensor_parallel_size=tensor_parallel_size,
)




/home/setup/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-20 08:50:11,642	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 09-20 08:50:11 utils.py:727] Gemma 2 uses sliding window attention for every odd layer, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 09-20 08:50:11 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='/data/2024_llm/sftlab/out_data/sftlab-experiments/test/1-gemma_full-zero3', speculative_config=None, tokenizer='/data/2024_llm/sftlab/out_data/sftlab-experiments/test/1-gemma_full-zero3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingCon

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.63it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.35it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.44it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.44it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.44it/s]



INFO 09-20 08:50:15 model_runner.py:1008] Loading model weights took 17.2179 GB
INFO 09-20 08:50:18 gpu_executor.py:122] # GPU blocks: 9987, # CPU blocks: 780
INFO 09-20 08:50:20 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-20 08:50:20 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-20 08:50:32 model_runner.py:1430] Graph capturing finished in 12 secs.


In [2]:

import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [3]:
import copy
def ask_model(messages,model):
    messages=copy.deepcopy(messages)

    if messages[-1]["role"]=="user":
        messages.append({"role": "assistant", "content": ""})
    else:
        messages.append({"role": "user", "content": ""})
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False,)
    prompt=prompt.replace("<end_of_turn>\n","")
    text=llm_gen(model, [prompt], temperature=0.7, top_k=50, max_tokens=1024)[0]
    return text
def llm_gen(model, prompt_list,
            temperature=0.7, top_k=50,
            max_tokens=1024,
            ):
    outputs = model.generate(
        prompt_list,
        sampling_params=SamplingParams(
            temperature=temperature,
            max_tokens=max_tokens,
            top_k=top_k,
            stop="<end_of_turn>",
        )
    )
    return [i.outputs[0].text.strip() for i in outputs]




In [4]:
messages = [
]
 
while True:
    q=input("input:")
    print("user: ",q)
    messages.append({"role": "user", "content": q})
    out=ask_model(messages,model)
    print("assistant: ",out)
    messages.append({"role": "assistant", "content": out})

user:  1+1は?


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.99it/s, est. speed input: 91.44 toks/s, output: 49.23 toks/s]


assistant:  <start_of_turn>user
2です。
user:  サイコロを二個ふって､合計が3以上になる確率は?


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  5.87it/s, est. speed input: 243.45 toks/s, output: 53.43 toks/s]


assistant:  <start_of_turn>user
5/6です。
user:  それはおかしくない?


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  5.32it/s, est. speed input: 317.12 toks/s, output: 53.74 toks/s]


assistant:  <start_of_turn>user
どうしたのか教えてくれる？
user:  


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.25it/s, est. speed input: 982.38 toks/s, output: 39.82 toks/s]


assistant:  もちろん。


KeyboardInterrupt: Interrupted by user

In [18]:
messages=[]
messages.append({"role": "user", "content": "やあ"})
messages.append({"role": "assistant", "content": "まあまあ"})
messages.append({"role": "user", "content": "やあ"})

In [19]:
messages=copy.deepcopy(messages)

if messages[-1]["role"]=="user":
    messages.append({"role": "assistant", "content": ""})
else:
    messages.append({"role": "user", "content": ""})

prompt = tokenizer.apply_chat_template(messages, tokenize=False,)
prompt=prompt.replace("<end_of_turn>\n","")

In [21]:
prompt

'<bos><start_of_turn>user\nやあ<start_of_turn>model\nまあまあ<start_of_turn>user\nやあ<start_of_turn>model\n'